TRY FLAG EMBEDDINGS FROM

https://github.com/FlagOpen/FlagEmbedding


https://github.com/FlagOpen/FlagEmbedding/blob/master/examples/pretrain/README.md


In [1]:
import shutil, os, subprocess
from google.colab import drive
drive.mount('/content/drive')

# LOAD DATA
import os, torch
# os.chdir('/content/drive/MyDrive/YouTube/')
!pwd

Mounted at /content/drive
/content


In [2]:
!pip install FlagEmbedding sentence-transformers jsonlines datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12

In [3]:
from datasets import load_dataset

dataset = load_dataset("scholarly360/indian_ipo_prospectus_data_with_pageno",split="test")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/38654 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3043 [00:00<?, ? examples/s]

Dataset({
    features: ['title_prospectus', 'href_prospectus', 'pdf_prospectus', 'content_page', 'page_no'],
    num_rows: 3043
})


In [4]:
dataset[0]

{'title_prospectus': 'AGS Transact Technologies Limited',
 'href_prospectus': 'https://www.sebi.gov.in/filings/public-issues/jan-2022/ags-transact-technologies-limited_55527.html',
 'pdf_prospectus': 'https://www.sebi.gov.in/sebi_data/attachdocs/jan-2022/1643088829506.pdf',
 'content_page': 'PROSPECTUS\nDated January 22, 2022\n\nPlease read Section 32 ofthe Companies Act, 2013\nBook Built Offer\n\n \n\nAGS TRANSACT TECHNOLOGIES LIMITED\n\n(Our Company was incorporated in Mumbai, Maharashtra on December 11, 2002 as AGS Infotech Private Limited, a private limited company under the Companies Act, 1956. The name of our Company was subsequently\nchanged to AGS Transact Technologies Private Limited and the Registrar of Companies, Maharashtra issued a fresh certificate of incorporation dated June 3, 2010. Our Company was then converted into a public limited\ncompany and consequently, its name was changed to AGS Transact Technologies Limited and the Registrar of Companies, Maharashtra issued a

In [5]:
def my_spliiter(full_string,x=5):
  ''' Split Large Chunk of Text into Array '''
  res=[full_string[y-x:y] for y in range(x, len(full_string)+x,x)]
  return(res)

print(my_spliiter("asdasfas",3))
print(my_spliiter("asdasfas",4))

['asd', 'asf', 'as']
['asda', 'sfas']


In [6]:
df = dataset.to_pandas()
df = df.head(500)
df

,title_prospectus,href_prospectus,pdf_prospectus,content_page,page_no
0,AGS Transact Technologies Limited,https://www.sebi.gov.in/filings/public-issues/...,https://www.sebi.gov.in/sebi_data/attachdocs/j...,"PROSPECTUS\nDated January 22, 2022\n\nPlease r...",0
1,AGS Transact Technologies Limited,https://www.sebi.gov.in/filings/public-issues/...,https://www.sebi.gov.in/sebi_data/attachdocs/j...,(This page is intentionally left blank)\n,1
2,AGS Transact Technologies Limited,https://www.sebi.gov.in/filings/public-issues/...,https://www.sebi.gov.in/sebi_data/attachdocs/j...,TABLE OF CONTENTS\n\nSECTION I: GENERAL...\n\n...,2
3,AGS Transact Technologies Limited,https://www.sebi.gov.in/filings/public-issues/...,https://www.sebi.gov.in/sebi_data/attachdocs/j...,SECTION I: GENERAL\nDEFINITIONS AND ABBREVIATI...,3
4,AGS Transact Technologies Limited,https://www.sebi.gov.in/filings/public-issues/...,https://www.sebi.gov.in/sebi_data/attachdocs/j...,\n\nTerm\n\nDescription\n\n \n\n \n\nESOS 201...,4
...,...,...,...,...,...
495,MedPlus Health Services Limited - Prospectus,https://www.sebi.gov.in/filings/public-issues/...,https://www.sebi.gov.in/sebi_data/attachdocs/d...,\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n ...,18
496,MedPlus Health Services Limited - Prospectus,https://www.sebi.gov.in/filings/public-issues/...,https://www.sebi.gov.in/sebi_data/attachdocs/d...,\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n ...,19
497,MedPlus Health Services Limited - Prospectus,https://www.sebi.gov.in/filings/public-issues/...,https://www.sebi.gov.in/sebi_data/attachdocs/d...,\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n ...,20
498,MedPlus Health Services Limited - Prospectus,https://www.sebi.gov.in/filings/public-issues/...,https://www.sebi.gov.in/sebi_data/attachdocs/d...,\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n ...,21


In [7]:
my_list = []
for index, row in df.iterrows():
  content_page = row['content_page']
  my_para_list = my_spliiter(content_page, 800)
  for sample_para in my_para_list:
    my_list.append({"text":sample_para})

In [8]:
print('len(my_list) :: ', len(my_list))
my_list[0]

len(my_list) ::  2375


{'text': 'PROSPECTUS\nDated January 22, 2022\n\nPlease read Section 32 ofthe Companies Act, 2013\nBook Built Offer\n\n \n\nAGS TRANSACT TECHNOLOGIES LIMITED\n\n(Our Company was incorporated in Mumbai, Maharashtra on December 11, 2002 as AGS Infotech Private Limited, a private limited company under the Companies Act, 1956. The name of our Company was subsequently\nchanged to AGS Transact Technologies Private Limited and the Registrar of Companies, Maharashtra issued a fresh certificate of incorporation dated June 3, 2010. Our Company was then converted into a public limited\ncompany and consequently, its name was changed to AGS Transact Technologies Limited and the Registrar of Companies, Maharashtra issued a fresh certificate of incorporation dated July 20, 2010. For further details in\nrelation to changes in th'}

# TRAIN

In [9]:
import jsonlines

with jsonlines.open('toy_pretrain_data.jsonl', 'w') as writer:
    writer.write_all(my_list)

In [10]:
! torchrun --nproc_per_node 1 -m FlagEmbedding.baai_general_embedding.retromae_pretrain.run \
--output_dir mod_2 \
--model_name_or_path BAAI/bge-base-en \
--train_data toy_pretrain_data.jsonl \
--learning_rate 2e-5 \
--num_train_epochs 3 \
--per_device_train_batch_size 4 \
--dataloader_drop_last True \
--max_seq_length 512 \
--logging_steps 100

2024-06-19 21:48:11.365845: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 21:48:11.365899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 21:48:11.489725: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-19 21:48:11.724888: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 21:48:12.845726: W tensorflow/comp

# TEST (Casual Here, Do a Proper STS Test)

In [11]:
sent_1 = """ Investments in equity and equity-related securities
involve a degree of risk and investors should not
invest any funds in the Offer unless they can afford
to take the risk of losing their entire investment.
"""
sent_2 = """
Investors are advised to read the risk factors
carefully before taking an investment
decision in the Offer.
"""

In [12]:
from sentence_transformers import SentenceTransformer

sentences = [sent_1,sent_2]
model = SentenceTransformer('BAAI/bge-base-en')
embeddings_1 = model.encode(sentences, normalize_embeddings=True)
embeddings_2 = model.encode(sentences, normalize_embeddings=True)
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[1.         0.87791663]
 [0.87791663 1.        ]]


In [13]:
from sentence_transformers import SentenceTransformer

sentences = [sent_1, sent_2]
model = SentenceTransformer('mod_2/encoder_model')
embeddings_1 = model.encode(sentences, normalize_embeddings=True)
embeddings_2 = model.encode(sentences, normalize_embeddings=True)
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

Some weights of BertModel were not initialized from the model checkpoint at mod_2/encoder_model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[[1.         0.85436887]
 [0.85436887 1.        ]]


In [14]:
sent_1 = """ Unless the context requires otherwise, the financial
information in this Prospectus is derived from the
Restated Consolidated Financial Statements of our
Corporation
"""
sent_2 = """
Risk Factors – Significant differences exist
between Indian GAAP and other accounting
principles, such as U.S. GAAP and IFRS and we prefer Consolidated Financial Statement made by Indian GAAP
"""

In [15]:
sentences = [sent_1,sent_2]
model = SentenceTransformer('BAAI/bge-base-en')
embeddings_1 = model.encode(sentences, normalize_embeddings=True)
embeddings_2 = model.encode(sentences, normalize_embeddings=True)
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

[[0.9999999 0.8042306]
 [0.8042306 1.0000001]]


In [16]:
sentences = [sent_1,sent_2]
model = SentenceTransformer('mod_2/encoder_model')
embeddings_1 = model.encode(sentences, normalize_embeddings=True)
embeddings_2 = model.encode(sentences, normalize_embeddings=True)
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

Some weights of BertModel were not initialized from the model checkpoint at mod_2/encoder_model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[[0.9999999  0.76391256]
 [0.76391256 1.        ]]
